<a href="https://colab.research.google.com/github/jaunty0216/Final_Project_forNTNU/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E5%B0%88%E9%A1%8C%E3%80%80%E3%80%90%E4%BA%8C%E6%9C%88%E3%80%91%E6%83%85%E7%B7%92%E6%8A%92%E7%99%BC%E5%B0%8F%E5%A4%A9%E5%9C%B0%E3%80%80%E6%96%87%E5%AD%97%E9%9B%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

【二月】情緒抒發小天地文字探勘

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('').sheet1

In [ ]:
dicts = gc.open_by_url('').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,時間戳記,這個月有什麼煩惱呢？
0,2025/5/1 上午 1:01:39,期中讀不完
1,2025/5/1 上午 1:02:07,期中停修了必修課．．．
2,2025/5/1 上午 1:02:31,讀書讀得很躁壓力很大
3,2025/5/1 上午 1:02:39,吃不飽
4,2025/5/1 上午 1:02:50,躺不平


In [ ]:
df.describe()

,時間戳記,這個月有什麼煩惱呢？
count,8,8
unique,8,8
top,2025/5/1 上午 1:01:39,期中讀不完
freq,1,1


In [ ]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 50.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=3355531096f5cda24badfeb178f2f427a95c326c0558ab3672132ea2fdbfe496
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [ ]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [ ]:
for _, row in dicts.iterrows():
    word = row['這個月有什麼煩惱呢？']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.951 seconds.
DEBUG:jieba:Loading model cost 1.951 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [ ]:
df['Chinese_Content'] = df['這個月有什麼煩惱呢？'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [ ]:
df['Chinese_Content']

,Chinese_Content
0,期中讀不完
1,期中停修了必修課
2,讀書讀得很躁壓力很大
3,吃不飽
4,躺不平
5,捲不贏
6,排球打不好
7,改選沒有過


In [ ]:
df['Tokenized']

,Tokenized
0,[期中讀不完]
1,"[期中, 停修, 了, 必修, 課]"
2,[讀書讀得很躁壓力很大]
3,[吃不飽]
4,[躺不平]
5,[捲不贏]
6,[排球打不好]
7,"[改選, 沒有過]"


In [ ]:
word_counts = Counter(all_words)
word_counts

Counter({'期中讀不完': 1,
         '期中': 1,
         '停修': 1,
         '必修': 1,
         '讀書讀得很躁壓力很大': 1,
         '吃不飽': 1,
         '躺不平': 1,
         '捲不贏': 1,
         '排球打不好': 1,
         '改選': 1,
         '沒有過': 1})

In [ ]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個', '得'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [ ]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:
word_freq_df

,Word,Frequency
0,期中讀不完,1
1,期中,1
2,停修,1
3,必修,1
4,讀書讀得很躁壓力很大,1
5,吃不飽,1
6,躺不平,1
7,捲不贏,1
8,排球打不好,1
9,改選,1


In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 2s (3,171 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [ ]:
from google.colab import userdata
apikey = userdata.get('key')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言主要反映了五月初使用者們普遍的**焦慮和壓力**情緒 (負面情緒)。

具體內容集中在學業和個人表現上：期中考試的壓力（讀不完、停修必修課、讀書壓力大、捲不贏），以及其他方面的挫折感（吃不飽、躺不平、排球打不好、改選失敗）。  這些內容都指向一種無力感和對自身能力的懷疑。  雖然留言內容不同，但它們都體現了使用者們在五月初正面臨著多重困境，並因此感到沮喪和焦慮。

因此，發文的風向是**負面且充滿壓力**的。

